# Install requirements 

In [1]:
# Install requirements
!pip install fastapi==0.68.1
!pip install opencv-python==4.5.3.56
# !pip install Pillow==8.3.2
!pip install timm==0.4.12
!pip install python-multipart==0.0.5
!pip install uvicorn==0.15.0

     |████████████████████████████████| 52 kB 834 kB/s 
     |████████████████████████████████| 10.9 MB 9.5 MB/s 
     |████████████████████████████████| 60 kB 7.9 MB/s 
     |████████████████████████████████| 49.9 MB 64 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 376 kB 5.2 MB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=346c065305499f2cfa14a126309e57a15e535e60f2d6437789be6f7b6c857483
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c

In [2]:
!pip install nest-asyncio

In [3]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=9ddc9a2b1d29378591b655ed01aceb752d7cc7d610a4b7c54757d10d2bde5886
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [4]:
!pip install validators matplotlib

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import cv2
import io
import numpy as np

import torch
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

## DPT Model

In [7]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [8]:
def load_model_dpt(model_type):
  ## Load model
  
  # MiDaS v3 - Large
  # (highest accuracy, slowest inference speed)
  model_type = "nvidia_resneXt"  
  
  # MiDaS v3 - Hybrid
  # (medium accuracy, medium inference speed)
  # model_type = "DPT_Hybrid"
  
  # (lowest accuracy, highest inference speed)
  # model_type = "MiDaS_small"  # MiDaS v2.1 - Small
  
  resneXt = torch.hub.load('vldcreation/corn-disease', model_type)
  utils = torch.hub.load('vldcreation/corn-disease', model_type)

  resneXt.eval().to(device)

  return resneXt  


def pre_process_dpt(image, model_type):

    corn_tranforms = torch.hub.load("vldcreation/corn-disease", "transforms")
    if model_type == "nvidia_resneXt":
        transform = corn_tranforms.dpt_transform
    else:
        transform = corn_tranforms.small_transfor

    # Load image
    img = cv2.imdecode(np.frombuffer(image.file.read(),
                                      np.uint8),
                        cv2.IMREAD_COLOR)

    # convert it to the correct format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Transform it so that it can be used by the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_batch = transform(img).to(device)

    # Return this image so it can be used in postprocessing
    return input_batch, img

def post_process_dpt(original, prediction):

  prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=original.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

  output = prediction.cpu().numpy()
  # Create a figure using matplotlib which super-imposes the original
  # image and the prediction

  fig = Figure()
  canvas = FigureCanvas(fig)
  ax = fig.gca()

  # Render both images original as foreground
  ax.imshow(original)
  ax.imshow(output)

  ax.axis("off")
  canvas.draw()

  # Reshape output to be a numpy array
  width, height = fig.get_size_inches() * fig.get_dpi()
  width = int(width)
  height = int(height)
  output_image = np.frombuffer(canvas.tostring_rgb(),
                                dtype='uint8').reshape(height, width, 3)

  # Encode to png
  res, im_png = cv2.imencode(".png", output_image)

  return im_png



# Setup the server

In [9]:

model_type = "nvidia_resneXt"
model = load_model_dpt(model_type)

# Code from: https://fastapi.tiangolo.com/tutorial/request-files/
app = FastAPI()


@app.post("/uploadfiles/")
async def create_upload_files(files: List[UploadFile] = File(...)):
    """ Create API endpoint to send image to and specify
     what type of file it'll take

    :param files: Get image files, defaults to File(...)
    :type files: List[UploadFile], optional
    :return: A list of png images
    :rtype: list(bytes)
    """

    for image in files:

        # Return preprocessed input batch and loaded image
        input_batch, image = pre_process_dpt(image, model_type)

        # Run the model and postpocess the output
        with torch.no_grad():
            prediction = model(input_batch)

        # # Post process and stitch together the two images to return them
        output_image = post_process_dpt(image, prediction)
            
        return StreamingResponse(io.BytesIO(output_image.tobytes()),
                                 media_type="image/png")


@app.get("/")
async def main():
    """Create a basic home page to upload a file

    :return: HTML for homepage
    :rtype: HTMLResponse
    """

    content = """<body>
          <h3>Upload an image to get it's depth map from the MiDaS model</h3>
          <form action="/uploadfiles/" enctype="multipart/form-data" method="post">
              <input name="files" type="file" multiple>
              <input type="submit">
          </form>
      </body>
      """
    return HTMLResponse(content=content)


HTTPError: ignored

In [ ]:
auth_token = "26l1O7acs0bIyop3pT0bQU7kcIA_4aYQWZW15m28SDkNCgNTz" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")

In [ ]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)

In [ ]:
# Check if it exists
!ps aux | grep ngrok

# Make magic happen

In [ ]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)